In [1]:
import pandas as pd
import sklearn as sklearn
import numpy as np
import time
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lg
import optuna
import optuna.integration.lightgbm as lgb
import xgboost as xgb
import pickle
import os
from datetime import date

from typing import List, TypeVar, Dict
import abc

from sklearn.pipeline import Pipeline
from pandas.core.frame import DataFrame
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import RepeatedKFold
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, roc_auc_score, precision_score, average_precision_score
from sklearn.metrics import classification_report, confusion_matrix
from kydavra import FisherSelector

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

## Helper functions

In [2]:
_СORRELATION_MATRIX_PATH_ = 'corr_matrix.pcl'
_СHURN_PATH_ = 'churn_model.pcl'
_CHURN_PATH_REDUCED_ = 'churn_model_reduced.pcl' 
_MAX_FEATURES_ = 100

In [3]:
# https://towardsdatascience.com/interpreting-roc-curve-and-roc-auc-for-classification-evaluation-28ec3983f077
# https://neptune.ai/blog/f1-score-accuracy-roc-auc-pr-auc
class Metrics:
    
    def roc_auc(y_true, predicted):
        return roc_auc_score(y_true, predicted)
    
    def auc(y_true, predicted):
        return average_precision_score(y_true, predicted)
    
    def classification_report(y_true, predicted):
        return classification_report(y_true, predicted)
    
    def confusion_matrix(y_true, predicted):
        return confusion_matrix(y_true, predicted)

In [4]:
def get_float_cols(df:DataFrame) -> List[str]:
    return df.select_dtypes(include=float).columns.tolist()

def get_int_cols(df:DataFrame) -> List[str]:
    return df.select_dtypes(include=int).columns.tolist()

def get_number_cols(df:DataFrame) -> List[str]:
    return df.select_dtypes(np.number).columns.tolist()

def get_obj_cols(df:DataFrame) -> List[str]:
    return list(df.select_dtypes(include=object).columns)

def print_empty_values(df:DataFrame):
    col_names_with_na = list(df.isna().sum()[lambda x: x > 0].index)
    col_names_with_empty = list(df.isnull().sum()[lambda x: x > 0].index)
    result = set(col_names_with_na) | set(col_names_with_empty)    
    print('Columns with NA or empty: {0}'.format(result))
    
def get_empty_cols(df:DataFrame):
    return list(df.isnull().sum()[lambda x: x > 0].index)

In [5]:
def reduce_mem_usage(df, verbose=True) -> pd.DataFrame:
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
def split_test_train(df:DataFrame):
    test, train = df[df['ind'].eq('test')], df[df['ind'].eq('train')]
    test = test.drop(['ind'], axis=1)
    train = train.drop(['ind'], axis=1)
    return test, train
    
def combine_test_train(test:DataFrame, train:DataFrame):
    combine = pd.concat([test.assign(ind='test'), train.assign(ind='train')])
    target = train['target']
    test_ids = test['Id']
    return combine, target, test_ids

def combine_data(train_df, train_num, train_dpi, test_df, test_num, test_dpi):
    df_combine = pd.concat([train_df.assign(ind='train'), test_df.assign(ind='test')])
    df_combine_num = pd.concat([train_num.assign(ind='train'), test_num.assign(ind='test')])
    df_combine_dpi = pd.concat([train_dpi.assign(ind='train'), test_dpi.assign(ind='test')])
    return df_combine, df_combine_num, df_combine_dpi

In [7]:
def reduce_churn_data():

    if os.path.exists(_CHURN_PATH_REDUCED_) == False:
        if os.path.exists(_СHURN_PATH_) == True: 
    
            with open(_СHURN_PATH_, 'rb') as file:
                deserialized_object = pickle.load(file)

            deserialized_object = list(deserialized_object)
        
            # reduce size
            train_df= reduce_mem_usage(deserialized_object[1][1])
            train_num_reduced = reduce_mem_usage(deserialized_object[1][2])
            train_dpi_reduced = reduce_mem_usage(deserialized_object[1][3])

            test_df = reduce_mem_usage(deserialized_object[2][1])
            test_num_reduced = reduce_mem_usage(deserialized_object[2][2])
            test_dpi_reduced = reduce_mem_usage(deserialized_object[2][3])
            
            # dump data back
            deserialized_object = tuple([[train_df, train_num_reduced, train_dpi_reduced], [test_df, test_num_reduced, test_dpi_reduced]])
            pickle.dump(deserialized_object, open(_CHURN_PATH_REDUCED_, 'wb'))
    else:
        print(f'{_CHURN_PATH_REDUCED_} already exist')

In [8]:
def load_churn_data():

    with open(_СHURN_PATH_, 'rb') as file:
        deserialized_object = pickle.load(file)

    train_df = deserialized_object[1][1]
    train_num = deserialized_object[1][2]
    train_dpi = deserialized_object[1][3]

    test_df = deserialized_object[2][1]
    test_num = deserialized_object[2][2]
    test_dpi = deserialized_object[2][3]

    return train_df, train_num, train_dpi, test_df, test_num, test_dpi

In [9]:
def load_churn_reduced():

    with open(_CHURN_PATH_REDUCED_, 'rb') as file:
        deserialized_object = pickle.load(file)

    train_df = deserialized_object[0][0]
    train_num = deserialized_object[0][1]
    train_dpi = deserialized_object[0][2]

    test_df = deserialized_object[1][0]
    test_num = deserialized_object[1][1]
    test_dpi = deserialized_object[1][2]

    return train_df, train_num, train_dpi, test_df, test_num, test_dpi

In [10]:
def is_short_number(number:str) -> bool:
    if (number.isdigit() and len(number) <= 4):
        return True
    return False

def is_life(number:str) -> bool:
    if (len(number) == 12 and (number[2:5] in ['063', '093' ])):
        return True
    return False

def is_kyivstar(number:str) -> bool:
    if (len(number) == 12 and (number[2:5] in ['067', '097', '068', '098'])):
        return True
    return False

### Feature selectors

In [11]:
class FeatureHelper:
    
    def get_feature_correlation_df(corr_m, remove_duplicates=True, remove_self_correlations=True):
    
        corr_matrix_abs = corr_m.abs()
        corr_matrix_abs_us = corr_matrix_abs.unstack()
        sorted_correlated_features = corr_matrix_abs_us \
            .sort_values(kind="quicksort", ascending=False) \
            .reset_index()

        # Remove comparisons of the same feature
        if remove_self_correlations:
            sorted_correlated_features = sorted_correlated_features[
                (sorted_correlated_features.level_0 != sorted_correlated_features.level_1)
            ]

        # Remove duplicates
        if remove_duplicates:
            sorted_correlated_features = sorted_correlated_features.iloc[:-2:2]

        # Create meaningful names for the columns
        sorted_correlated_features.columns = ['f1', 'f2', 'corr']

        return sorted_correlated_features
    
    def get_correlation_matrix(df:DataFrame, method:str, save_path:str):
        if os.path.exists(save_path) == False:
            corr_matrix = df.corr(method = method, numeric_only = True)
            pickle.dump(corr_matrix, open(save_path, 'wb'))
        else:
            corr_matrix = pickle.load(open(save_path, 'rb'))

        return corr_matrix

    def remove_aggr_function(str_to_check:str) -> str:
        parts = str_to_check.split('_')
        
        if (len(parts) > 2):
            index_to_remove = len(parts) - 2
            
            # remove aggregation function
            if (parts[index_to_remove] in ['min', 'std', 'max', 'mea', 'td']):
                parts.remove(parts[index_to_remove])
                
            result = '_'.join(parts)
            return result
        else:
            return str_to_check    

        
    def get_heatmap_matrix(corr_matrix:DataFrame):
        heatmap_matrix = pd.DataFrame(corr_matrix['target'].abs())
        heatmap_matrix = heatmap_matrix.sort_values(by='target', ascending=False)
        heatmap_matrix = heatmap_matrix.drop(index=['target'])           
        return heatmap_matrix
    
    # index - column name
    # target - value
    def plot_heatmap(heatmap_matrix:DataFrame):
        plt.figure(figsize=(40, 120))
        heatmap = sns.heatmap(heatmap_matrix, vmin=-1, vmax=1, annot=True, cmap='BrBG')
        heatmap.set_title('Features Correlating with Churn Rate', fontdict={'fontsize':18}, pad=16);
        return heatmap_matrix
    
    def get_important_features(heatmap_matrix:DataFrame, use_groupping = False, num_of_features:int = -1):
        df_features = heatmap_matrix.reset_index()
        df_features = df_features.rename(columns = {'index':'feature'})
        
        # apply aggregation function for further groupping
        df_features['feature_group'] = df_features['feature'].apply(FeatureHelper.remove_aggr_function)
        df_features = df_features[['feature', 'feature_group', 'target']]
        sorted_features = df_features.sort_values(by=['feature_group', 'target'], ascending = [False, False])
        
        # take first item from the group
        if (use_groupping == True):
            important_features = sorted_features.groupby('feature_group').first()
        else:
            important_features = sorted_features

        # order by target
        important_features = important_features.sort_values(by='target', ascending=False)
           
        # take N first rows
        if (num_of_features != -1):
            important_features = important_features.head(num_of_features)
        
        # optimize for heatmap
        important_features = important_features.reset_index()
        important_features = important_features[['feature', 'target']]
        important_features.index = important_features['feature']
        important_features.index.name = None
        important_features = important_features[['target']]
        important_features = important_features[important_features['target'] > 0]
        
        return important_features
    
    def get_important_features_tuples(heatmap_matrix:DataFrame, num_of_features:int = -1):
        important_features = FeatureHelper.get_important_features(heatmap_matrix, num_of_features)
        
        if (num_of_features == -1):
            num_of_features = len(important_features)
        
        important_features_tuples = list(zip(important_features.index, 
                                             important_features.target, 
                                             list(range(0, num_of_features))))
        
        return important_features_tuples

In [12]:
class FeatureSelector(abc.ABC):

    @abc.abstractmethod
    def get_heatmap(self) -> pd.DataFrame:
        pass
    
    @abc.abstractmethod
    def plot_heatmap(self):
        pass

    @abc.abstractmethod
    def get_important_features(self) -> pd.DataFrame:
        pass
    
    @abc.abstractmethod
    # 1 - feature name, 2 - target, 3 - sorted number
    def get_important_features_tuples(self) -> List[tuple[str, float, int]]:
        pass

In [13]:
class CorrelationSelector(FeatureSelector):
    
    def __init__(self, data:pd.DataFrame, corr_method:str, num_of_features:int):
        self.data = data
        self.corr_method = corr_method
        self.file_prefix = corr_method
        self.num_of_features = num_of_features
    
    def get_heatmap(self) -> pd.DataFrame:
        self.save_path = f'{self.file_prefix}_{_СORRELATION_MATRIX_PATH_}'
        self.corr_m = FeatureHelper.get_correlation_matrix(self.data, self.corr_method, self.save_path)
        self.heatmap_m = FeatureHelper.get_heatmap_matrix(self.corr_m)
        return self.heatmap_m 
    
    def get_important_features(self) -> pd.DataFrame:
        return FeatureHelper.get_important_features(self.heatmap_m, self.num_of_features)
    
    def plot_heatmap(self):
        FeatureHelper.plot_heatmap(self.get_important_features())
    
    def get_non_correlated_features(self, barrier_coef:float, do_log:bool) -> List[str]:
        
        important_tuples = FeatureHelper.get_important_features_tuples(self.heatmap_m, self.num_of_features)
        # f1, f2, corr
        features_corr = FeatureHelper.get_feature_correlation_df(self.corr_m)
        
        already_processed = set()
        all_features = [t[0] for t in important_tuples]

        for f in all_features:
            # get correlated features
            correlated = list(features_corr[(features_corr['f1']==f) & (features_corr['corr'] > barrier_coef)]['f2'])

            # if highly correlated features exist
            if (len(correlated)>0):

                for to_remove in correlated:
                    if (to_remove not in already_processed):
                        if (to_remove in all_features):
                            all_features.remove(to_remove)
                            if do_log: print(f'Removing: {to_remove} for {f}')            

        # remember initial feature
        already_processed.add(f)
        
        # return non-correlated features
        return all_features
    
    def get_important_noncorrelated_features_tuples(self, mutual_correlation:float, feature_importance:float):
        
        final = []
        important = self.get_important_features_tuples()
        non_correlated = self.get_non_correlated_features(mutual_correlation, False)
        
        for i in important:
            if (i[0] in non_correlated and i[1] > feature_importance):
                final.append(i)

        return final
                        
    def get_important_features_tuples(self) ->  List[tuple[str, float, int]]:
        return FeatureHelper.get_important_features_tuples(self.heatmap_m, self.num_of_features)
    
    def __str__(self):
        return type(self).__name__ + '_' + self.corr_method

In [14]:
# https://nitin9809.medium.com/lightgbm-binary-classification-multi-class-classification-regression-using-python-4f22032b36a2
# https://www.analyticsvidhya.com/blog/2020/10/feature-selection-techniques-in-machine-learning/
class LGBMSelector(FeatureSelector):
    
    def __init__(self, data:pd.DataFrame, num_of_features:int):
        self.data = data
        self.num_of_features = num_of_features
        self.train_model()
    
    def train_model(self):
        test, train = split_test_train(self.data)

        self.y_train = train['target'].round(0).astype(int)
        self.y_test = test['target'].round(0).astype(int)

        train = train.drop(['target'], axis=1)
        test = test.drop(['target'], axis=1)

        # save regressor
        self.regressor = lg.LGBMClassifier()
        self.regressor.fit(train, self.y_train)
        predicted = self.regressor.predict(test)
        
        # save predicted data
        self.predicted = predicted
    
    def get_heatmap(self) -> pd.DataFrame:
        df_feature_importance = pd.DataFrame(list(zip(list(self.regressor.feature_importances_), list(self.data.columns))))
        df_feature_importance = df_feature_importance.set_axis(['target', 'feature'], axis=1)
        df_feature_importance = df_feature_importance.sort_values(by=['target'], ascending=False)
        df_feature_importance = df_feature_importance.set_index('feature')
        df_feature_importance.index.name = None
        self.heatmap_m = df_feature_importance
        return self.heatmap_m
    
    def plot_heatmap(self):
        FeatureHelper.plot_heatmap(self.get_important_features())

    def get_important_features(self) -> pd.DataFrame:
        return FeatureHelper.get_important_features(self.heatmap_m, self.num_of_features)
    
    def get_important_features_tuples(self) -> List[tuple[str, float, int]]:
        features_tuples = FeatureHelper.get_important_features_tuples(self.heatmap_m, self.num_of_features)
        return features_tuples
        
    def get_ROCAUC(self):
        return Metrics.roc_auc(self.y_test, self.predicted)

    def get_confusion_matrix(self):
        return Metrics.confusion_matrix(self.y_test, self.predicted)
    
    def get_classification_report(self):
        return Metrics.classification_report(self.y_test, self.predicted)
    
    def __str__(self):
        return type(self).__name__

In [15]:
# https://neptune.ai/blog/lightgbm-parameters-guide
class EnhancedLGBMSelector(LGBMSelector):
    
    def train_model(self):        
        
        test, train = split_test_train(df_combine)

        self.y_train = train['target'].round(0).astype(int)
        self.y_test = test['target'].round(0).astype(int)

        self.x_train = train.drop(['target'], axis=1)
        self.x_test = test.drop(['target'], axis=1)

        # Specifying the parameter
        d_train = lgb.Dataset(x_train, label=y_train)
        d_test = lgb.Dataset(x_test, label=y_test, reference=d_train)

        # load model from disk
        if os.path.exists(_СHURN_PATH_) == True: 
            with open('LGBM_model_800_features.pcl', 'rb') as file:
                self.regressor = pickle.load(file)
        else:
            params={}
            params['boosting_type']='gbdt' 
            params['objective']='binary' 
            params['metric']='auc' 
            params['verbosity'] = 0

            #train the model 
            self.regressor=lgb.train(params = params, 
                          train_set = d_train,
                          valid_sets = d_test, 
                          num_boost_round = 20, 
                          callbacks= [lgb.early_stopping(stopping_rounds=10)])
        
        #prediction on the test set
        self.predicted = self.regressor.predict(x_test)


In [16]:
class TransformPipe:
    
    def __init__(self, funcs, **kwargs):
        self.funcs = funcs
        self.kwargs = kwargs
    
    def transform(self, df:DataFrame) -> DataFrame:
        for f in self.funcs:
            df = f(df, **self.kwargs)
            
        return df

## Load Data

In [17]:
train_df, train_num, train_dpi, test_df, test_num, test_dpi = load_churn_reduced()
df_combine, df_combine_num, df_combine_dpi = combine_data(train_df, train_num, train_dpi, test_df, test_num, test_dpi)

### Feature importance

In [18]:
# https://towardsdatascience.com/deep-dive-on-ml-techniques-for-feature-selection-in-python-part-2-c258f8a2ac43
# https://www.kaggle.com/code/gomes555/tps-jun2021-feature-selection-lightgbm-tuner

result = {}
seleсtors = [ CorrelationSelector(df_combine, 'pearson', -1),
              CorrelationSelector(df_combine, 'spearman', -1) ]

for selector in seleсtors:
    heatmap = selector.get_heatmap()
    features = selector.get_important_features()
    features_tuples = selector.get_important_features_tuples()
    result[selector.__str__()] = selector

In [19]:
# normal = LGBMSelector(df_combine, -1)

In [20]:
# enhanced = EnhancedLGBMSelector(df_combine, -1)

In [21]:
# normal.get_ROCAUC(), enhanced.get_ROCAUC()

In [22]:
test, train = split_test_train(df_combine)

y_train = train['target'].round(0).astype(int)
y_test = test['target'].round(0).astype(int)

x_train = train.drop(['target'], axis=1)
x_test = test.drop(['target'], axis=1)

#Specifying the parameter
d_train = lgb.Dataset(x_train, label=y_train)
d_test = lgb.Dataset(x_test, label=y_test, reference=d_train)

params={}
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='binary' #Binary target feature
params['metric']='auc' #metric for binary classification
#params['num_boost_round']=100
#params['early_stopping_rounds']=30
#params['max_depth']=100
#params['is_unbalance']=True
params['verbosity'] = 2

#train the model 
clf=lgb.train(params = params, 
              train_set = d_train,
              valid_sets = d_test, 
              num_boost_round = 20, 
              callbacks= [lgb.early_stopping(stopping_rounds=10)])

#prediction on the test set
predicted=clf.predict(x_test)


[I 2023-07-01 00:28:47,743] A new study created in memory with name: no-name-88c2c11e-f6e1-4583-839a-02341f78fa75
feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 9586, number of negative: 140414
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.880361
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.515813
[LightGBM] [Debug] init for col-wise cost 0.355101 seconds, init for row-wise cost 1.666131 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.950028 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 123224
[LightGBM] [Info] Number of data points in the train set: 150000, number of used features: 784
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.063907 -> initscore=-2.684292
[LightGBM] [Info] Start training from score -2.684292
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
Training until validation scores don't improve for 10 rounds
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 

feature_fraction, val_score: 0.883539:  14%|#4        | 1/7 [00:31<03:08, 31.39s/it]

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
Did not meet early stopping. Best iteration is:
[20]	valid_0's auc: 0.883539
[LightGBM] [Info] Number of positive: 9586, number of negative: 140414
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.880361
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.515813
[LightGBM] [Debug] init for col-wise cost 0.291937 seconds, init for row-wise cost 1.293693 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.515542 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 123224
[LightGBM] [Info] Number of data points in the train set: 150000, number of used features: 784
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.063907 -> initscore=-2.684292
[LightGBM] [Info] Start training from score -2.684292
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree 

feature_fraction, val_score: 0.886879:  29%|##8       | 2/7 [01:00<02:29, 29.87s/it]

...

[I 2023-07-01 01:36:54,631] Trial 66 finished with value: 0.8911021565714513 and parameters: {'min_child_samples': 50}. Best is trial 66 with value: 0.8911021565714513.
min_data_in_leaf, val_score: 0.891102:  80%|########  | 4/5 [36:56<14:17, 857.11s/it]

[LightGBM] [Info] Number of positive: 9586, number of negative: 140414
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.880361
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.515813
[LightGBM] [Debug] init for col-wise cost 0.451700 seconds, init for row-wise cost 1.470046 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.713202 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 123224
[LightGBM] [Info] Number of data points in the train set: 150000, number of used features: 784
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.063907 -> initscore=-2.684292
[LightGBM] [Info] Start training from score -2.684292
[LightGBM] [Debug] Trained a tree with leaves = 115 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 115 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 115 and depth = 12
[LightGBM] [Debug] Trained a tree with le

min_data_in_leaf, val_score: 0.891102: 100%|##########| 5/5 [37:40<00:00, 563.94s/it][I 2023-07-01 01:37:38,715] Trial 67 finished with value: 0.8911021565714513 and parameters: {'min_child_samples': 100}. Best is trial 66 with value: 0.8911021565714513.


[LightGBM] [Debug] Trained a tree with leaves = 115 and depth = 15
Did not meet early stopping. Best iteration is:
[20]	valid_0's auc: 0.891102


min_data_in_leaf, val_score: 0.891102: 100%|##########| 5/5 [37:40<00:00, 452.10s/it]


In [23]:
Metrics.roc_auc(y_test, predicted)

0.8911021565714514

In [33]:
fff.predict(x_test)

array([0.0237755 , 0.01348663, 0.01358502, ..., 0.0397863 , 0.04060678,
       0.59270895])

In [32]:
Metrics.roc_auc(y_test, predicted)

0.6224238253867485

In [24]:
existing = ['target', 'ind']
roc_auc = -1
prev_roc_auc = -1
increase_rate = -1

for c in list(df_combine.columns):
    if c != 'target':
        
        # add column
        existing.append(c)
        
        # new dataset
        df = df_combine[existing]
        
        # build model
        test, train = split_test_train(df)
        y_train = train['target']
        y_test = test['target']
        train = train.drop(['target'], axis=1)
        test = test.drop(['target'], axis=1)
        regressor = lg.LGBMClassifier()
        regressor.fit(train, y_train)
        
        # calculate metric
        predicted = regressor.predict(test)
        
        try:
            roc_auc = Metrics.roc_auc(y_test, predicted)
        except ValueError:
            roc_auc = -1
        
        print(f'{len(existing)} -> {roc_auc}')
        
        # remove current column
        # existing 

3 -> 0.5
4 -> 0.5
5 -> 0.5000247244095933
6 -> 0.49999288747270565
7 -> 0.49999288747270565
8 -> 0.562710535166727
9 -> 0.562710535166727
10 -> 0.5629195075422255
11 -> 0.5748062123180953
12 -> 0.575639392736126
13 -> 0.575575549426414
14 -> 0.5756463358274835
15 -> 0.5754974810621133
16 -> 0.5757315167190786
17 -> 0.5737078370499589
18 -> 0.5737078370499589
19 -> 0.5744663359313992
20 -> 0.5789362128999035
21 -> 0.5808645511942971
22 -> 0.5804355987804852
23 -> 0.5770969506326258
24 -> 0.5760161872467884
25 -> 0.5779761930421329
26 -> 0.5780649301973749
27 -> 0.5765264254166746
28 -> 0.5765264254166746
29 -> 0.5782031161718039
30 -> 0.5787951484055748
31 -> 0.5786925250676175
32 -> 0.5782246231896236
33 -> 0.5788908286521744
34 -> 0.5775121862996596
35 -> 0.5775121862996596
36 -> 0.5768918733926481
37 -> 0.5787846490505701
38 -> 0.5816553953204356
39 -> 0.581336178771875
40 -> 0.5796738825072713
41 -> 0.5792945483485827
42 -> 0.5801063911847305
43 -> 0.5796456018340308
44 -> 0.5823569

KeyboardInterrupt: 

### Check if dataset is balanced

In [ ]:
churned = len(df_combine[df_combine['target'] == 1])
not_churned = len(df_combine[df_combine['target'] == 0])

In [ ]:
y = [churned, not_churned]
x = ['Churned', 'Not churned']
plt.bar(x,y)
plt.show()

## Feature engineering
#### Check inbound calls from non-vodaphone number
#### Check outbound calls to non-vodaphone number
#### SMS from non-vodaphone number
#### SMS to non-vodaphone number

## Correlation matrix

## Explore numbers abonent had communication with + frequency

In [ ]:
churned = df_combine[df_combine['target'] == 1]
churned_with_nums = pd.merge(churned, df_combine_num, on='abon_id', how='left')
churned_numbers = list(churned_with_nums['bnum'].unique())

non_churned = df_combine[df_combine['target'] == 0]
non_churned_with_nums = pd.merge(non_churned, df_combine_num, on='abon_id', how='left')
non_churned_numbers = list(non_churned_with_nums['bnum'].unique())

number_abon_had_communicated = (set(churned_numbers) - set(non_churned_numbers))
df_number_abon_had_communicated = pd.DataFrame(number_abon_had_communicated, columns= ['bnum'])

In [ ]:
churned_with_nums[churned_with_nums['bnum'].isin(list(number_abon_had_communicated))]

## Telephone Feature

## Groupping

In [ ]:
gr = train_num.groupby(['abon_id'])
gr.groups

In [ ]:
# cor_matrix = train_df.corr()